### Case Study 4 : Financial Delinquency

Submitted by:

- Ravi Sivaraman
- Balaji Avvaru
- Apurv Mittal

In [1]:
from scipy.io import arff
import time
import pandas as pd
import numpy as np
import os
from os.path import isfile, join
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve, accuracy_score, confusion_matrix, average_precision_score
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn import metrics as mt
import getpass
from sklearn.model_selection import cross_validate, cross_val_predict, StratifiedKFold, GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

In [3]:
current_user = getpass.getuser()

if current_user == 'balaj':
    data_path = "/Users/balaj/OneDrive/Desktop/Docs/Docs 1/SMU/MSDS 7333/Case Study 4/Data"
elif current_user == 'ravis':
    data_path = "/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/Case Study 4/Data"
elif current_user == "apurv":
    data_path = "/Users/apurv/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/SMU/7333 - QTW/Case Study 4/Data"

# get all data files
data_files = [f for f in os.listdir(data_path) if (not f.startswith('.')) and os.path.isfile(join(data_path, f))]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ravis/Documents/GitHub/QTW-CaseStudy4/Data'

In [ ]:
data_files

In [ ]:
df = pd.DataFrame()

for f in data_files:
    data_temp = arff.loadarff(os.getcwd()+'/Data/'+f)
    temp_df = pd.DataFrame(data_temp[0])
    df = df.append(temp_df, ignore_index=True)


In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

##### Missing value analysis

In [ ]:
# Validate null values in the csv file
df.isnull().sum().sum()

In [ ]:
pd.set_option('display.max_rows', 100)
# Validate null values in the csv file
print((df.isnull().sum()).sort_values(ascending=False))
# print columns with null values
missing_data_columns = df.columns[df.isnull().any()]
print("\n\nColumns with null values")
print("************************")
missing_data_columns

In [ ]:
# percentage of missing values in each variable
(df[missing_data_columns].isnull().sum()/len(df)*100).sort_values(ascending=False)

In [ ]:
top5_missing_data_col = ['Attr37', 'Attr21', 'Attr27', 'Attr60', 'Attr45']
df[top5_missing_data_col].describe()

In [ ]:
#fill NA with median() of each column in dataset
df = df.apply(lambda x: x.fillna(x.median()),axis=0)

In [ ]:
# Validate null values in the csv file
df.isnull().sum().sum()

##### Independent Variable analysis

In [ ]:
#Visualizing the hist of data to check normality of independent variable
df_X = df.drop(['class'],axis=1)
df_X.hist(bins=50,figsize=(25,30))
plt.show()

In [ ]:
#heatmap - correlation matrix
plt.figure(figsize=(55, 50)) #code reference (5-1)
sns.heatmap(df_X.corr(), annot=True)
plt.title('HeatMap-Correlation Matrix')

##### Check for Multicolliniarity 

In [ ]:
#https://www.projectpro.io/recipes/drop-out-highly-correlated-features-in-python
# to drop features with colliniarity more than 95%
pd.set_option('display.max_rows', 100)

corr_df = pd.DataFrame(df_X.corr().abs())
corr_df.head(100)

In [ ]:
# Multi Colliniarity analysis on Independent variables 
upper_tri = corr_df.where(np.triu(np.ones(corr_df.shape),k=1).astype(np.bool))
print(upper_tri)

In [ ]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9999)]
print((to_drop))

#### Target

In [ ]:
df['class'] = df['class'].replace([b'0', b'1'], [0, 1])

df['class'].value_counts()

In [ ]:
sns.countplot(x = "class", data = df)
plt.title("Distribution of Target Values")
plt.show()

In [ ]:
# Pie chart
df['class'].value_counts().plot.pie(autopct = "%.1f%%")
plt.title("Proportion of Target Value")
plt.show()

In [ ]:
X = df.drop(['class'],axis=1)
ind_columns = df.drop('class',axis=1).columns
y = df['class']

We did normalize the attributes using StandardScaler() to scale them between 0 and 1 before running models.

In [ ]:
# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

We chose a stratified k-fold validation algorithm. In stratified k-fold cross-validation, the original sample is randomly partitioned into k equal size subsamples in which each fold contains roughly the same proportions of class labels. Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k-1 subsamples are used as training data. The cross-validation process is then repeated k times (the folds), with each of the k subsamples used exactly once as the validation data. The k results from the folds can then be averaged (or otherwise combined) to produce a single estimation. The advantage of this method is that all observations are used for both training and validation, and each observation is used for validation exactly once.

The typical standard of 10 folds will be adequate for this dataset

In [ ]:
#Create Cross Validation Procedure
cv = StratifiedKFold(n_splits=10, random_state=1234, shuffle=True)

#### helper functions

In [ ]:
# Model Metrics
def displayModel_metrics(best_model, grid_model, features, target, cv):   
    start = time.time()
    cv_results = cross_validate(best_model, features, target, cv=cv, scoring=['accuracy','precision','recall','f1'], n_jobs=-1)
    elapsed_time = (time.time() - start) 
    print ('Fold Scores:')
    print(' ')
    print(cv_results['test_accuracy'])
    print(' ')
    print('Best Accuracy   :  {:.3f}'.format(grid_model.best_score_))
    print('Mean Accuracy   : ', cv_results['test_accuracy'].mean())
    print('Mean Precision  : ', cv_results['test_precision'].mean())
    print('Mean Recall     : ', cv_results['test_recall'].mean())
    print('Mean F-Score   : ', cv_results['fscore'].mean())
    print('Mean Fit Time   : ', cv_results['fit_time'].mean())
    print('Mean Score Time : ', cv_results['score_time'].mean())
    print('CV Time         : ', elapsed_time)
    return

# ROC curve plot
def roc_curve_plot(model_fit, features, target):

    sns.set_palette("dark")

    yhat_score = model_fit.predict_proba(features)

    # Compute ROC curve for a subset of interesting classes
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in np.unique(target):
        fpr[i], tpr[i], _ = mt.roc_curve(y, yhat_score[:, i], pos_label=i)
        roc_auc[i] = mt.auc(fpr[i], tpr[i])

    for i in np.unique(target):
        plt.plot(fpr[i], tpr[i], label= ('class %d (area = %0.2f)' % (i, roc_auc[i])))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

    plt.legend(loc="lower right")  
    plt.title('Receiver operating characteristic')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()    

#### Model 1: Random Forest with default parameters

In [ ]:
rf_clf = RandomForestClassifier(class_weight='balanced', random_state=1234)
rf_clf.fit(X_scaled, y)

rf_clf.get_params()

In [ ]:
y_hat = rf_clf.predict(X_scaled)
accuracy_score(y_hat, y)

In [ ]:
confusion_matrix(y, y_hat)

In [ ]:
# ROC curve for Random Forest Classifier
roc_curve_plot(rf_clf, X_scaled, y)

In [ ]:
disp = plot_precision_recall_curve(rf_clf, X_scaled, y)
disp.ax_.set_title('Precision-Recall Curve')

In [ ]:
# cross validation
cv_df = pd.DataFrame(cross_validate(rf_clf, X_scaled, y, cv=cv, scoring=['accuracy','precision','recall', 'f1'], n_jobs=-1))
cv_df

#### Model 2: Random Forest with GridSearch

Random forest is an ensemble tree-based learning algorithm where it combines more than one algorithms of same or different kind for classifying objects. The Random Forest Classifier is a set of decision trees from randomly selected subset of training set. It aggregates the votes from different decision trees to decide the final class of the test object.

Parameters:

- n_estimators: number of trees in the forest

- max_depth: max number of levels in each decision tree

- criterion: The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. Note: this parameter is tree-specific

- min_samples_split = min number of data points placed in a node before the node is split

- min_samples_leaf = min number of data points allowed in a leaf node

- class_weight: The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))

In [ ]:
RF = RandomForestClassifier()

# define parameters       
max_depth_RF = [5, 7, 8, 10, 12]
random_state_RF = [1234]
n_estimators_RF =  [100]
criterion_RF = ['entropy']
min_samples_leaf_RF = [3, 4, 5]
min_samples_split_RF = [8, 10, 12]
class_weight_RF = ['balanced']

# define grid search
# param_grid_RF = dict(n_estimators=n_estimators_RF, max_depth=max_depth_RF, random_state=random_state_RF,
#                      criterion=criterion_RF, min_samples_leaf=min_samples_leaf_RF,
#                     min_samples_split=min_samples_split_RF, class_weight=class_weight_RF)

# search_RF = GridSearchCV(estimator=RF, param_grid=param_grid_RF, n_jobs=3, cv=cv, 
#                                scoring='accuracy',error_score=0, verbose=1)


# define random search
param_random_RF = dict(n_estimators=n_estimators_RF, max_depth=max_depth_RF, random_state=random_state_RF,
                     criterion=criterion_RF, min_samples_leaf=min_samples_leaf_RF,
                    min_samples_split=min_samples_split_RF, class_weight=class_weight_RF)


search_RF = RandomizedSearchCV(estimator=RF, param_distributions=param_random_RF, n_jobs=3, cv=cv, 
                               scoring='accuracy',n_iter=20, verbose=5)

In [ ]:
%%time
result_RF = search_RF.fit(X_scaled, y)
# summarize results
print("Best: %f using %s" % (result_RF.best_score_, result_RF.best_params_))
means = result_RF.cv_results_['mean_test_score']
stds = result_RF.cv_results_['std_test_score']
params = result_RF.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# # The GridSearch algorithm determined the following optimal parameters
best_Estimator_RF =result_RF.best_estimator_
Coef_weights_RF = result_RF.best_estimator_.feature_importances_
best_Estimator_RF

In [ ]:
# Display model metrics
displayModel_metrics(best_Estimator_RF, result_RF, X_scaled, y, cv)

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(best_Estimator_RF, X_scaled, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)

In [ ]:
conf_mat

In [ ]:
# ROC curve for Random Forest Classifier
roc_curve_plot(result_RF, X_scaled, y)

In [ ]:
disp = plot_precision_recall_curve(best_Estimator_RF, X_scaled, y)
disp.ax_.set_title('Precision-Recall Curve')

In [ ]:
from sklearn import metrics
precision, recall, threshold = metrics.precision_recall_curve(y, best_Estimator_RF.predict_proba(X_scaled)[:, 1])

In [ ]:

%matplotlib inline
plt.plot(threshold, precision[:-1], label='Precision')
plt.plot(threshold, recall[:-1], label='Recall')
plt.xlabel('Thresold')
plt.ylabel('Proportion')
plt.legend()

If we wanted to ensure that the model classify 95% of the financial institutions that go bankrupt  correctly we would want to select the following threshold:

In [ ]:
mal95_ind = np.argmin(recall >= 0.95)-1
mal95_thresh = threshold[mal95_ind]
mal95_precision = precision[mal95_ind]
mal95_recall = recall[mal95_ind]

print("Threshold:", mal95_thresh)
print("Precision:", mal95_precision)
print("Recall:", mal95_recall)

#### Feature Importance

In [ ]:
# Important features with their weights 
imp_feature_df = pd.DataFrame({'feature_names':ind_columns, 
                               'Coef_weights':Coef_weights_RF})
imp_feature_df.sort_values(by='Coef_weights', inplace=True, ascending=False )

imp_feature_df

In [ ]:
# Visulization of important features 
%matplotlib inline

ax = sns.barplot(x ='Coef_weights', y = 'feature_names',data=imp_feature_df.head(10), orient= 'h')
ax.set_title("Random Forest Feature Importance")
ax.set_xlabel("Coefficient Magnitude\n(z-score)")
ax.set_ylabel("Feature Names")

#### Model 3: XGBoost with default parameters  

In [ ]:
xgb_clf = xgb.XGBClassifier(random_state=1234)

In [ ]:
xgb_clf.fit(X_scaled, y)

xgb_clf.get_params()

In [ ]:
y_hat = xgb_clf.predict(X_scaled)
accuracy_score(y_hat, y)

In [ ]:
confusion_matrix(y, y_hat)

In [ ]:
# ROC curve for XGB Classifier
roc_curve_plot(xgb_clf, X_scaled, y)

In [ ]:
disp = plot_precision_recall_curve(xgb_clf, X_scaled, y)
disp.ax_.set_title('Precision-Recall Curve')

In [ ]:
# cross validation
cv_df = pd.DataFrame(cross_validate(xgb_clf, X_scaled, y, cv=cv, scoring=['accuracy','precision','recall', 'f1'], n_jobs=-1))
cv_df

#### Model 4: XGBoost with GridSearch

Parameters

- learning_rate: The learning rate. In each boosting step, this values shrinks the weight of new features, preventing overfitting or a local minimum. This value must be between 0 and 1. The default value is 0.3.

- max_depth: The maximum depth of a tree. Be careful, greater the depth, greater the complexity of the model and more easy to overfit. This value must be an integer greater than 0 and have 6 as default.

- n_estimators: The number of trees in our ensemble.

- gamma: A regularization term and it’s related to the complexity of the model. It’s the minimum loss necessary to occur a - -split in a leaf. It can be any value greater than zero and has a default value of 0.

- colsample_bytree: Represents the fraction of columns to be subsampled. It’s related to the speed of the algorithm and prevent overfitting. Default value is 1 but it can be any number between 0 and 1.

- lambda: L2 regularization on the weights. This encourages smaller weights. Default is 1 but it can be any value.

In [ ]:
XGB = xgb.XGBClassifier()

# define parameters       
clf_n_estimators_XGB = [200]
# clf_learning_rate_XGB =  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5]   0.3
# clf_max_depth_XGB = range(3, 15) 5
# clf_colsample_bytree_XFB = [i/10.0 for i in range(1, 3)]  0.2
# clf_gamma_XGB = [i/10.0 for i in range(1, 8)]          0.1
# lambda_XGB = [0.1, 1.0, 5.0, 10.0, 50.0, 100.0]        0.1
# min_child_weight = [0.1, 0.9, 0.95]                    1

clf_learning_rate_XGB =  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5]	 
clf_max_depth_XGB = range(3, 15)						 
clf_colsample_bytree_XFB = [i/10.0 for i in range(1, 3)]		 
clf_gamma_XGB = [0.01, 0.05, 0.1, 0.2, 0.3]				 
lambda_XGB = [0.01, 0.05, 0.1, 1.0, 5.0, 10.0, 50.0, 100.0]		 
min_child_weight = [0.1, 0.9, 0.95, 2, 3]	
random_state_XGB = [1234]

# define grid search
# param_grid_RF = dict(n_estimators=clf_n_estimators_XGB, learning_rate=clf_learning_rate_XGB, 
#                      max_depth=clf_max_depth_XGB, colsample_bytree = clf_colsample_bytree_XFB,
#                     gamma=clf_gamma_XGB, reg_lambda=lambda_XGB)

# search_RF = GridSearchCV(estimator=RF, param_grid=param_grid_RF, n_jobs=3, cv=cv, 
#                                scoring='accuracy',error_score=0, verbose=1)


# define random search
param_random_XGB = dict(n_estimators=clf_n_estimators_XGB, learning_rate=clf_learning_rate_XGB, 
                     max_depth=clf_max_depth_XGB, colsample_bytree = clf_colsample_bytree_XFB,
                    gamma=clf_gamma_XGB, reg_lambda=lambda_XGB, random_state=random_state_XGB)


search_XGB = RandomizedSearchCV(estimator=XGB, param_distributions=param_random_XGB, n_jobs=3, cv=cv, 
                               scoring='accuracy',n_iter=20, verbose=5)

In [ ]:
%%time
result_XGB = search_XGB.fit(X_scaled, y)
# summarize results
print("Best: %f using %s" % (result_XGB.best_score_, result_XGB.best_params_))
means = result_XGB.cv_results_['mean_test_score']
stds = result_XGB.cv_results_['std_test_score']
params = result_XGB.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# # The GridSearch algorithm determined the following optimal parameters
best_Estimator_XGB =result_XGB.best_estimator_
Coef_weights_XGB = result_XGB.best_estimator_.feature_importances_
best_Estimator_XGB

In [ ]:
# Display model metrics
displayModel_metrics(best_Estimator_XGB, result_XGB, X_scaled, y, cv)

In [ ]:
# ROC curve for Random Forest Classifier
roc_curve_plot(result_XGB, X_scaled, y)

In [ ]:
disp = plot_precision_recall_curve(best_Estimator_XGB, X_scaled, y)
disp.ax_.set_title('Precision-Recall Curve')

In [ ]:
precision, recall, threshold = metrics.precision_recall_curve(y, best_Estimator_XGB.predict_proba(X_scaled)[:, 1])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(threshold, precision[:-1], label='Precision')
plt.plot(threshold, recall[:-1], label='Recall')
plt.xlabel('Thresold')
plt.ylabel('Proportion')
plt.legend()

If we wanted to ensure that the model classify 95% of the financial institutions that go bankrupt  correctly we would want to select the following threshold:

In [ ]:
mal95_ind = np.argmin(recall >= 0.95)-1
mal95_thresh = threshold[mal95_ind]
mal95_precision = precision[mal95_ind]
mal95_recall = recall[mal95_ind]

print("Threshold:", mal95_thresh)
print("Precision:", mal95_precision)
print("Recall:", mal95_recall)

### Feature Importance with XGB

In [ ]:
# Important features with their weights 
imp_feature_df = pd.DataFrame({'feature_names':ind_columns, 
                               'Coef_weights':Coef_weights_XGB})
imp_feature_df.sort_values(by='Coef_weights', inplace=True, ascending=False )

imp_feature_df

In [ ]:
# Visulization of important features 
%matplotlib inline

ax = sns.barplot(x ='Coef_weights', y = 'feature_names',data=imp_feature_df.head(10), orient= 'h')
ax.set_title("XGB Feature Importance")
ax.set_xlabel("Coefficient Magnitude\n(z-score)")
ax.set_ylabel("Feature Names")